# 전처리된 데이터 불러오기

- 앞서 전처리가 완료된 데이터를 가져와 RNN 모델에 학습시켜 성능을 평가해보자.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
train_data = pd.read_csv("./Preprocessed/train_data_preprocessed.csv")
test_data = pd.read_csv("./Preprocessed/test_data_preprocessed.csv")
with open("./Preprocessed/data_configs.json", "r") as json_file:
    prepro_configs = json.load(json_file)

In [3]:
# 학습 데이터
train_input = np.load(open("./Preprocessed/train_input_embedded.npy", "rb"))
train_label = np.array(train_data.label)

# 테스트 데이터
test_input = np.load(open("./Preprocessed/test_input_embedded.npy", "rb"))
test_label = np.array(test_data.label)

word2vec_weights = np.load(open("./Preprocessed/word2vec_weights.npy", "rb"))

# 하이퍼 파라미터 정의

RNN의 하이퍼 파라미터를 설정해주자. 

In [4]:
# Hyperparameters for model training
BATCH_SIZE = 256
NUM_EPOCHS = 10
VALID_SPLIT = 0.1

# Hyperparameters for model layers
kwargs = {"input_dimension": word2vec_weights.shape[0],
        "embedding_dimension": word2vec_weights.shape[1],
        "input_length": 15,
        "dropout_rate": 0.1,
        "lstm_dimension": 64,
        "dense_dimension": 32,
        "output_dimension": 1}

# 모델 구조 정의

- 임베딩 레이어 --> 드랍아웃 레이어 --> 양방향 LSTM 레이어 --> 드랍아웃 레이어 --> 두 개의 fully connected 레이어

In [5]:
import tensorflow as tf

class RNNClassifier(tf.keras.Model):
    
    # __init__ method
    def __init__(self, **kwargs):
        super(RNNClassifier, self).__init__()  # super() is used to inherit parent's variables 
        self.embedding = tf.keras.layers.Embedding(input_dim = kwargs["input_dimension"], 
                                          output_dim = kwargs["embedding_dimension"],
                                          input_length = kwargs["input_length"],
                                          weights = [word2vec_weights])
        self.lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(kwargs["lstm_dimension"]))
        self.dropout = tf.keras.layers.Dropout(kwargs["dropout_rate"])
        self.fc1 = tf.keras.layers.Dense(units = kwargs["dense_dimension"], 
                                         activation = tf.keras.activations.relu)
        self.fc2 = tf.keras.layers.Dense(units = kwargs["output_dimension"], 
                                         activation = tf.keras.activations.sigmoid)
     
    
    # call method to run layers
    def call(self, x):
        x = self.embedding(x)      # word embedding layer
        x = self.dropout(x)        # dropout layer
        x = self.lstm_layer(x)     # pass LSTM layer
        x = self.dropout(x)        # dropout layer
        x = self.fc1(x)            # dense layer
        x = self.fc2(x)            # output layer (binary classification)
        
        return x

# 모델 생성 및 피팅

이제 학습 데이터를 넣어서 모델을 훈련시켜보자.

학습 속도를 높이기 위해서 구글 코랩에서 GPU를 이용해 모델을 훈련시키고, 구해진 가중치를 불러와 모델을 평가하는 과정을 거치도록 하겠다.

In [6]:
# Create rnn model
rnn = RNNClassifier(**kwargs)  

# Comile model
rnn.compile(optimizer = tf.keras.optimizers.RMSprop(0.001),                # optimizer
           loss = tf.keras.losses.BinaryCrossentropy(),                    # loss function
           metrics = [tf.keras.metrics.BinaryAccuracy(name="accuracy")])   # evaluation metric

In [ ]:
import os

# EarlyStopping
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", min_delta = 0.001, patience = 3)

# ModelCheckpoint
checkpoint_path = "./nsmc_rnn.h5"
    
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor="val_accuracy", 
                                   verbose=1, save_best_only=True, save_weights_only=False)

# Train the model
history = rnn.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                 validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

- 학습 시간이 오래걸리는 것을 고려해서 위의 코드를 직접 돌리지는 않을 것이다. 
- 훈련되지 않은 모델의 정확도와 훈련된 가중치들을 불러와 정의한 모델의 정확도를 비교해보자.

---
학습 과정에 대한 그래프: `pd.DataFrame(history.history).plot(xlabel="Epochs")`

<div>
<img src="history.png" width="400", height="200", align="left">
</div>

# 모델 성능 평가

In [7]:
loss, acc = rnn.evaluate(test_input,  test_label, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1547/1547 - 4s - loss: 0.6948 - accuracy: 0.4995
훈련되지 않은 모델의 정확도: 49.95%


In [8]:
# 가중치 로드
rnn.load_weights("./nsmc_rnn.h5")

# 모델 재평가
loss, acc = rnn.evaluate(test_input,  test_label, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1547/1547 - 2s - loss: 0.3495 - accuracy: 0.8551
복원된 모델의 정확도: 85.51%


- 구글 코랩을 이용해 훈련된 가중치를 불러와 테스트 데이터를 바탕으로 성능을 평가해보니 약 85.5%의 정확도를 보이는 것을 확인할 수 있었다.